In [1]:
from random import sample
from time import time
import pandas as pd
import pymongo
from sklearn import ensemble
import numpy as np
import os
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

<h3><u>CONSTANTS AND HELPER FUNSTIONS</u></h3>

In [2]:
query_collection = "noaa_nam_2"
mongo_url = "mongodb://lattice-100:27018/"
mongo_db_name = "sustaindb"
query_fild = "gis_join"
sample_percent = 0.1
train_test = 0.8
feature_importance_percentage = 98
exhaustive_sample_percent = 0.0001


training_labels = ["mean_sea_level_pressure_pascal",
                   "surface_pressure_surface_level_pascal",
                   "10_metre_u_wind_component_meters_per_second",
                   "10_metre_v_wind_component_meters_per_second",
                   "soil_temperature_kelvin"]

target_labels = ["pressure_pascal"]

# QUERY-RELATED
sustainclient = pymongo.MongoClient(mongo_url)
sustain_db = sustainclient[mongo_db_name]

# QUERY projection
client_projection = {}
for val in training_labels:
    client_projection[val] = 1
for val in target_labels:
    client_projection[val] = 1
    
    
def fancy_logging(msg, unique_id=""):
    print(unique_id, ":", "====================================")
    print(unique_id, ":", msg, ": TIME: ",time())


<h3><u>DATA FETCH</u></h3>

In [3]:
# ACTUAL QUERYING
def query_sustaindb(query_gisjoin):

    sustain_collection = sustain_db[query_collection]
    client_query = {query_fild: query_gisjoin}

    start_time = time()
    query_results = list(sustain_collection.find(client_query, client_projection))
    
    return list(query_results)

def queryall_sustaindb():

    sustain_collection = sustain_db[query_collection]
    client_query = {}

    start_time = time()
    query_results = list(sustain_collection.find(client_query, client_projection))
    
    return list(query_results)

In [4]:
#df = query_sustaindb('G3701310')
df = queryall_sustaindb()
print("1: ", len(df))

1:  7431587


<h3><u>DATA SAMPLING</u></h3>

In [5]:
def data_sampling(query_results, exhaustive, sample_percent=1):
    if exhaustive:
        all_data = query_results
    else:
        data_size = int(len(query_results) * sample_percent)
        all_data = sample(query_results, data_size)

    return pd.DataFrame(all_data)

In [6]:
# RIKI
sampled_df = data_sampling(df, False, exhaustive_sample_percent)

In [7]:
Y = sampled_df.loc[:,target_labels]
X = sampled_df.loc[:, training_labels]
print(X.shape, Y.shape)

(743, 5) (743, 1)


<h3><u>DATA SPLITTING INTO TRAING AND VALIDATION</u></h3>

In [8]:
'''def data_partitioning(query_results, exhaustive, sample_percent=1):
    if exhaustive:
        all_data = query_results
    else:
        data_size = int(len(query_results) * sample_percent)
        all_data = sample(query_results, data_size)

    msk = np.random.rand(len(all_data)) < train_test

    all_data = pd.DataFrame(all_data)
    training_data = all_data[msk]
    val_data = all_data[~msk]
    return (pd.DataFrame(training_data), pd.DataFrame(val_data))'''

'def data_partitioning(query_results, exhaustive, sample_percent=1):\n    if exhaustive:\n        all_data = query_results\n    else:\n        data_size = int(len(query_results) * sample_percent)\n        all_data = sample(query_results, data_size)\n\n    msk = np.random.rand(len(all_data)) < train_test\n\n    all_data = pd.DataFrame(all_data)\n    training_data = all_data[msk]\n    val_data = all_data[~msk]\n    return (pd.DataFrame(training_data), pd.DataFrame(val_data))'

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(594, 5) (149, 5) (594, 1) (149, 1)


<h3><u>MODELING</u></h3>

In [10]:
'''parameters = [ {'n_estimators': 500, 'max_depth': 2, 'min_samples_split': 20},
                {'n_estimators': 300, 'max_depth': 3, 'min_samples_split': 50},
              {'n_estimators': 500, 'max_depth': 3, 'min_samples_split': 20},
              {'n_estimators': 600, 'max_depth': 3, 'min_samples_split': 15}]
    
for params in parameters:
    print("PARAMETERS:",params)
    count = 0
    error = 0
    for i in range(0,5):
        print("ROUND:",i)
        clf = ensemble.RandomForestRegressor(**params)
        clf.fit(X_train, pd.Series.ravel(y_train))

        rmse = sqrt(mean_squared_error(pd.Series.ravel(y_test), clf.predict(X_test)))
        print(rmse)
        error = error + rmse

        feature_importance = clf.feature_importances_
        feature_importance = 100.0 * (feature_importance / feature_importance.sum())
        sorted_idx = np.argsort(feature_importance)
        count = count+1
        print(np.flip(sorted_idx), np.flip(feature_importance[sorted_idx]))
    
    print("===============================================================")

    print("AVG RMSE:",(error/count))'''

# BETTER ALTERNATIVE BELOW

'parameters = [ {\'n_estimators\': 500, \'max_depth\': 2, \'min_samples_split\': 20},\n                {\'n_estimators\': 300, \'max_depth\': 3, \'min_samples_split\': 50},\n              {\'n_estimators\': 500, \'max_depth\': 3, \'min_samples_split\': 20},\n              {\'n_estimators\': 600, \'max_depth\': 3, \'min_samples_split\': 15}]\n    \nfor params in parameters:\n    print("PARAMETERS:",params)\n    count = 0\n    error = 0\n    for i in range(0,5):\n        print("ROUND:",i)\n        clf = ensemble.RandomForestRegressor(**params)\n        clf.fit(X_train, pd.Series.ravel(y_train))\n\n        rmse = sqrt(mean_squared_error(pd.Series.ravel(y_test), clf.predict(X_test)))\n        print(rmse)\n        error = error + rmse\n\n        feature_importance = clf.feature_importances_\n        feature_importance = 100.0 * (feature_importance / feature_importance.sum())\n        sorted_idx = np.argsort(feature_importance)\n        count = count+1\n        print(np.flip(sorted_idx), np.

In [11]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
param_grid = {'max_depth': [2, 3], 'min_samples_split': [15, 20, 50]}
base_est = ensemble.RandomForestRegressor(random_state=0)
sh = HalvingGridSearchCV(base_est, param_grid, cv=5, verbose=3, 
                         factor=2, resource='n_estimators', max_resources=600).fit(X, pd.Series.ravel(Y))

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 150
max_resources_: 600
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 6
n_resources: 150
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END max_depth=2, min_samples_split=15, n_estimators=150;, score=(train=0.250, test=0.113) total time=   0.4s
[CV 2/5] END max_depth=2, min_samples_split=15, n_estimators=150;, score=(train=0.240, test=0.138) total time=   0.3s
[CV 3/5] END max_depth=2, min_samples_split=15, n_estimators=150;, score=(train=0.243, test=0.085) total time=   0.3s
[CV 4/5] END max_depth=2, min_samples_split=15, n_estimators=150;, score=(train=0.220, test=0.274) total time=   0.3s
[CV 5/5] END max_depth=2, min_samples_split=15, n_estimators=150;, score=(train=0.208, test=0.125) total time=   0.3s
[CV 1/5] END max_depth=2, min_samples_split=20, n_estimators=150;, score=(train=0.250, test=0.113) total time=   0.3s
[CV 2/5] END max_depth=2, min_samp

In [12]:
# THE BEST MODEL
clf_best = sh.best_estimator_

In [13]:
rmse = sqrt(mean_squared_error(pd.Series.ravel(y_test), clf_best.predict(X_test)))
rmse

3874.244869769391

<h3><u>EXTRACT TOP FEATURES</u></h3>

In [14]:
feature_importance = clf_best.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.sum())
sorted_idx = np.argsort(feature_importance)
print(np.flip(sorted_idx), np.flip(feature_importance[sorted_idx]))

feature_importance = np.flip(feature_importance[sorted_idx])
sorted_idx=np.flip(sorted_idx)

print(sorted_idx)
print(feature_importance)


[0 4 1 2 3] [37.80652075 34.2716431  13.82809946  7.27583804  6.81789865]
[0 4 1 2 3]
[37.80652075 34.2716431  13.82809946  7.27583804  6.81789865]


<h3><u>FIND N FOR WHICH IMPORTANCE % > feature-importance-percentage</u></h3>

In [15]:
def find_cumulative(lists, val_max):
    cu_list = []
    length = len(lists)
    cu_list = [sum(lists[0:x:1]) for x in range(1, length+1)]
    
    print(cu_list)
    res = next(x for x, val in enumerate(cu_list)
                                  if val > val_max)
    return res

In [16]:
cut_off_indx = find_cumulative(feature_importance, feature_importance_percentage)

print("LAST INDEX: ", cut_off_indx)

[37.806520748922814, 72.07816384533966, 85.9062633089815, 93.1821013529657, 99.99999999999999]
LAST INDEX:  4


In [17]:
chopped_indices = sorted_idx[0:cut_off_indx+1]

print(sorted_idx)
print(chopped_indices)

[0 4 1 2 3]
[0 4 1 2 3]


<h3><u>SELECTED TOP COLUMNS</u></h3>

In [18]:
candidate_x_columns = list(X.columns)
candidate_y_columns = list(Y.columns)

print(candidate_x_columns)
print(candidate_y_columns)

['mean_sea_level_pressure_pascal', 'surface_pressure_surface_level_pascal', '10_metre_u_wind_component_meters_per_second', '10_metre_v_wind_component_meters_per_second', 'soil_temperature_kelvin']
['pressure_pascal']


In [19]:
selected_x_columns = [candidate_x_columns[i] for i in chopped_indices]
selected_x_columns

['mean_sea_level_pressure_pascal',
 'soil_temperature_kelvin',
 'surface_pressure_surface_level_pascal',
 '10_metre_u_wind_component_meters_per_second',
 '10_metre_v_wind_component_meters_per_second']

<b><hr /></b>

<h1><u><b>TRAINING PHASE #2</b></u></h1>

<h3><u>AGGREGATE QUERY OVER THE CHOSEN COLUMNS PER GIS-JOIN</u></h3>

In [20]:
'''sustain_collection = sustain_db[query_collection]
pipeline=[
   { "$project": { 'gis_join': '$gis_join', 'max_specific_humidity': '$max_specific_humidity', 'max_max_air_temperature': '$max_max_air_temperature', 'max_min_air_temperature': '$max_min_air_temperature'}},
   { "$group": { '_id': "$gis_join", 
"avg_max_specific_humidity": { "$avg": "$max_specific_humidity" },
"avg_max_max_air_temperature": { "$avg": "$max_max_air_temperature" },
"avg_max_min_air_temperature": { "$avg": "$max_min_air_temperature" }
  } }
]
cur = sustain_collection.aggregate(pipeline)

results = list(cur)
len(results)
'''

'sustain_collection = sustain_db[query_collection]\npipeline=[\n   { "$project": { \'gis_join\': \'$gis_join\', \'max_specific_humidity\': \'$max_specific_humidity\', \'max_max_air_temperature\': \'$max_max_air_temperature\', \'max_min_air_temperature\': \'$max_min_air_temperature\'}},\n   { "$group": { \'_id\': "$gis_join", \n"avg_max_specific_humidity": { "$avg": "$max_specific_humidity" },\n"avg_max_max_air_temperature": { "$avg": "$max_max_air_temperature" },\n"avg_max_min_air_temperature": { "$avg": "$max_min_air_temperature" }\n  } }\n]\ncur = sustain_collection.aggregate(pipeline)\n\nresults = list(cur)\nlen(results)\n'

In [21]:
chopped_projection = []
chopped_projection.extend(selected_x_columns)
chopped_projection.extend(candidate_y_columns)

print(chopped_projection)

['mean_sea_level_pressure_pascal', 'soil_temperature_kelvin', 'surface_pressure_surface_level_pascal', '10_metre_u_wind_component_meters_per_second', '10_metre_v_wind_component_meters_per_second', 'pressure_pascal']


In [22]:
def construct_chopped_query(chopped_projection, gis_join):
    # PROJECTION
    proj_d = {}
    proj_dict = {'$project': proj_d}
    
    #GROUP + AGGREGATION
    group_d = {}
    group_dict = {'$group': group_d}
    
    full_query=[proj_dict, group_dict]
    
    # PROJECTION PART
    for cp in chopped_projection:
        proj_d[cp] = "$"+str(cp)
    proj_d[gis_join] = "$"+str(gis_join)
    
    # GROUP PART
    group_d['_id'] = "$"+str(gis_join)
    for cp in chopped_projection:
        inner_dict = {}
        inner_dict["$avg"] = "$"+str(cp)
        group_d[cp] = inner_dict
    
    return full_query
    
    

In [23]:
agg_pipeline = construct_chopped_query(chopped_projection, query_fild)

In [24]:
sustain_collection = sustain_db[query_collection]
cur = sustain_collection.aggregate(agg_pipeline)
agg_results = list(cur)

print(len(agg_results))

3065


In [25]:
agg_results[0]

{'_id': 'G2800830',
 'mean_sea_level_pressure_pascal': 102234.0659090909,
 'soil_temperature_kelvin': 277.1112470222242,
 'surface_pressure_surface_level_pascal': 101758.07575757576,
 '10_metre_u_wind_component_meters_per_second': -0.12724994457129277,
 '10_metre_v_wind_component_meters_per_second': 0.3196709069338712,
 'pressure_pascal': 22627.514105132133}

<h3><u>DATA STAGING FOR PHASE 2</u></h3>

In [26]:
phase2_df = pd.DataFrame(agg_results)

In [27]:
chopped_projection

['mean_sea_level_pressure_pascal',
 'soil_temperature_kelvin',
 'surface_pressure_surface_level_pascal',
 '10_metre_u_wind_component_meters_per_second',
 '10_metre_v_wind_component_meters_per_second',
 'pressure_pascal']

In [28]:
df_importance = phase2_df.loc[:, chopped_projection]
df_importance

,mean_sea_level_pressure_pascal,soil_temperature_kelvin,surface_pressure_surface_level_pascal,10_metre_u_wind_component_meters_per_second,10_metre_v_wind_component_meters_per_second,pressure_pascal
0,102234.065909,277.111247,101758.075758,-0.127250,0.319671,22627.514105
1,102246.208333,275.947863,100803.669823,-0.212209,0.086574,23359.711075
2,102276.465909,274.152724,99848.284091,0.176917,0.150258,23679.786832
3,101970.238444,274.115271,80418.955829,2.531348,-0.692039,23128.271681
4,102247.914983,277.273358,100544.394781,0.118152,0.214494,20937.194240
...,...,...,...,...,...,...
3060,102022.247934,276.984723,101871.529614,1.275100,0.172684,22423.450744
3061,101974.813740,268.399047,95431.540076,0.476142,1.340470,22165.048730
3062,100713.001193,280.044527,95611.141937,0.258859,6.306155,24948.541097
3063,102163.510101,279.646758,100528.098485,-0.114529,0.993724,22426.756529


<h3><u>K-MEANS CLUSTERING</u></h3>

In [29]:
num_clusters = int(sqrt(len(agg_results)))

num_clusters

55

In [30]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')


In [31]:
kmeans = KMeans(n_clusters=num_clusters).fit(df_importance)
centroids = kmeans.cluster_centers_
print(centroids)

[[ 1.02094183e+05  2.71126635e+02  9.83491463e+04  1.04481658e+00
   9.65980195e-01  2.30599910e+04]
 [ 1.01624498e+05  2.72477867e+02  8.55800281e+04 -2.27342203e-03
   2.26294882e+00  2.33334610e+04]
 [ 1.02129662e+05  2.80528724e+02  1.00577855e+05  6.01700489e-01
   4.76282264e-03  1.75947082e+04]
 [ 1.02048884e+05  2.70189431e+02  7.78161447e+04  1.35862908e+00
   1.06653056e+00  2.28694485e+04]
 [ 1.01818074e+05  2.73614308e+02  9.03345125e+04  1.44048031e+00
   7.66892354e-01  2.23091430e+04]
 [ 1.02199261e+05  2.72085537e+02  9.97048499e+04  5.92306883e-01
   6.68757893e-01  2.28456529e+04]
 [ 1.01910817e+05  2.72107941e+02  9.44087428e+04  7.09198887e-01
   9.43789965e-01  2.23015163e+04]
 [ 1.02208595e+05  2.85037018e+02  1.02048883e+05  6.67016988e-01
  -6.90333953e-01  1.08856458e+04]
 [ 1.02168170e+05  2.78705884e+02  1.01207482e+05  1.66612243e-01
   3.75058473e-01  2.08707957e+04]
 [ 1.02148912e+05  2.69303647e+02  7.28173794e+04  1.55507437e+00
   5.16517791e-01  2.2745

In [32]:
from sklearn.metrics import pairwise_distances_argmin_min

df_ultimate = pd.DataFrame(columns=["gis_join", "cluster_id", "distance"])



for index, row in phase2_df.iterrows():
    input_x = row[chopped_projection]
    gis_join = row['_id']
    #print(input_x, gis_join)
    closest, d = pairwise_distances_argmin_min([np.array(input_x)], centroids)
    df_ultimate.loc[index] = [gis_join, closest[0], d[0]]
    
print(df_ultimate)



      gis_join cluster_id     distance
0     G2800830         22   500.242358
1     G2800710         37   337.606283
2     G4701690         19   717.048229
3     G3500570         45   613.846118
4     G2800990          8   671.159339
...        ...        ...          ...
3060  G3700150         47   679.101926
3061  G4600110         36   283.439265
3062  G4100150         11  1509.798680
3063  G4804990         17   249.765275
3064  G3200310         23   687.283010

[3065 rows x 3 columns]


In [33]:
df_ultimate.to_csv("~/ucc-21/clusters-noaa.csv")